In [ ]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets
from IPython.display import display

class PointEditor:
    def __init__(self, filename):
        self.filename = filename
        self.original_data, self._mask = self.load_points(filename)
        self.data = self.original_data[self._mask]
        self.fig = self.create_figure()
        self.setup_ui()

    def load_points(self, filename):
        try:
            points = np.genfromtxt(filename, delimiter=',')
            mask = np.ones(len(points), dtype=bool)  # Private mask initialized
            return points, mask
        except IOError:
            print("File not found.")
            return None, None

    def create_figure(self):
        fig = go.FigureWidget([go.Scattergl(x=self.data[:, 0], y=self.data[:, 1], mode='markers',
                                            marker=dict(color='blue', size=1),
                                            selected=dict(marker=dict(color='red', size=1)),
                                            unselected=dict(marker=dict(color='blue', size=1)))])

        fig.update_layout(clickmode='event+select')
        fig.update_traces(marker_size=0.1)

        fig.update_layout(
            width=800,  # Set the width of the figure
            height=600,  # Set the height of the figure
            autosize=False,  # Disable autosizing to use the specified dimensions
            margin=dict(l=50, r=50, b=100, t=100, pad=4)  # Adjust margins if needed
        )

        # To set a fixed aspect ratio:
        fig.update_yaxes(
            scaleanchor="x",
            scaleratio=1,
        )

        return fig

    def delete_points(self, btn):
        inds = self.fig.data[0].selectedpoints
        if inds:
            # Translate selected indices to original data indices
            valid_inds = [i for i in inds if i < len(self.data)]
            if valid_inds:
                original_inds = np.where(self._mask)[0][valid_inds]
                self._mask[original_inds] = False  # Update the private mask
                self.data = self.original_data[self._mask]  # Filter data based on updated mask
                self.fig.data[0].x = self.data[:, 0]
                self.fig.data[0].y = self.data[:, 1]
                print(f"Deleted {len(valid_inds)} points.")
            else:
                print("No valid points selected for deletion.")
        else:
            print("No points selected.")

    def save_points(self, btn):
        np.savetxt("deleted_points_mask.csv", self._mask, delimiter=",", fmt="%s")
        print("Mask saved to 'deleted_points_mask.csv'.")

    def setup_ui(self):
        delete_button = widgets.Button(description="Delete Selected")
        delete_button.on_click(self.delete_points)

        save_button = widgets.Button(description="Save Data")
        save_button.on_click(self.save_points)

        display(widgets.HBox([delete_button, save_button]))
        display(self.fig)

# Example usage
editor = PointEditor('original_points.csv')
